In [ ]:
# import pandas as pd
# import yfinance as yf
# from tqdm import tqdm
# from time import sleep

# ticker_dict = pd.read_csv('./tickers.csv')
# ticker_dict

# historical_dataframes = []

# for index, row in tqdm(ticker_dict.iterrows(), total=len(ticker_dict), desc="Processing"):
#     company = row['name']
#     ticker = row['Ticker']

#     stock = yf.Ticker(ticker)

#     # Retrieve historical market data
#     hist = stock.history(period="max")  # You can adjust the period as needed

#     # Add ticker as a new column
#     hist['Ticker'] = ticker
#     hist['name'] = company
    
#     print(f"Retrieved {len(hist)} rows of data for {company} ({ticker})")
#     # Save historical data to list
#     historical_dataframes.append(hist)
#     sleep(1)  # Be nice to Yahoo Finance API

# # Combine all historical dataframes into one
# all_data = pd.concat(historical_dataframes)

# # Reset index to make it cleaner
# all_data.reset_index(inplace=True)
# all_data.head()

# all_data.to_csv('stock_data.csv.zip', index=False, compression='zip')

In [ ]:
# import pickle
# import pandas as pd
# import time
# from bs4 import BeautifulSoup
# import requests
# import time
# import pandas as pd
# from tqdm import tqdm
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.firefox.options import Options
# from io import StringIO
# # #read nct_ids.pkl
# # with open('nct_ids.pkl', 'rb') as f:
# #     nct_ids = pickle.load(f)

# studies = pd.read_csv('../CTTI/studies.txt', sep='|')
# interventions = pd.read_csv('../CTTI/interventions.txt', sep='|')

# interventions = interventions[interventions['intervention_type'].str.lower().isin(['drug', 'biological'])]
# studies = studies[studies['nct_id'].isin(interventions['nct_id'])]

# studies = studies[~studies['overall_status'].str.lower().isin(['terminated', 'withdrawn', 'suspended', 'withheld', 'no longer available', 'temporarily not available'])]

# studies = studies.dropna(subset=['phase'])

# chrome_options = Options()
# chrome_options.add_argument("--headless") #FOR DEBUG COMMENT OUT SO YOU CAN SEE WHAT YOU'RE DOING
# driver = webdriver.Firefox(options=chrome_options)

# amendment_counts = []
# for i, nct in enumerate(tqdm(studies['nct_id'])):
#     try:
#         driver.get(f'https://clinicaltrials.gov/study/{nct}?tab=history')
#         # driver.page_source # needs to be called before the next line
#         time.sleep(1)
#         card_content = driver.find_element("class name","card-content").get_attribute('innerHTML')

#         versions_df = pd.read_html(StringIO(card_content))[0]
#         latest_version = versions_df['Version'].iloc[-2]

#         amendment_counts.append([nct, latest_version])
#     except Exception as e:
#         print(f"Error for {nct}: {e}")
#     # break    
#     # Optional: Introduce a delay between requests
#     #time.sleep(1)

# # # beautify the output of https://clinicaltrials.gov/study/NCT02840448?tab=history
# # import requests
# # from bs4 import BeautifulSoup
# # response = requests.get('https://clinicaltrials.gov/study/NCT02840448?tab=history')
# # bs = BeautifulSoup(response.text, 'html.parser')
# # bs


In [31]:
#read sponsors data
import pandas as pd
import os
from tqdm import tqdm

def calculate_sma_slope(df, ticker, given_date, window_size, days):
    """
    Calculate the slope of Simple Moving Average (SMA) within a specified number of days from a given date for a given ticker.

    Parameters:
    - df: DataFrame containing historical stock prices with 'Date', 'Ticker', and 'Close' columns.
    - ticker: Ticker symbol of the stock.
    - given_date: Date for which to calculate the slope of SMA within the specified window.
    - window_size: Number of days for the SMA window.
    - days: number of days after the trial completion date

    Returns:
    - slope: Slope of SMA within the specified window from the given date for the given ticker.
    """
    # Convert given_date to a Timestamp object if it's in string format
    if isinstance(given_date, str):
        given_date = pd.to_datetime(given_date, utc=True)

    # Filter DataFrame for the given ticker
    df_ticker = df[df['name'] == ticker]

    if not isinstance(df_ticker['Date'].iloc[0], pd.Timestamp):
        df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], utc=True)

    df_ticker = df_ticker.sort_values(by='Date')

    selected_dates = df_ticker['Close'][df_ticker['Date'] > given_date]

    # Calculate SMA using rolling method only for dates after the given date
    sma = selected_dates.rolling(window=window_size, min_periods = 1).mean()

    # Calculate the slope of SMA within the specified window after the given date
    if len(sma) < days:
        return pd.NA
    slope = (sma.iloc[days-1] - sma.iloc[0]) / window_size

    return slope


CTTI_PATH = '../CTTI/'
SAVE_STOCKS_PATH = './stock_data.csv.zip'

sponsors = pd.read_csv(os.path.join(CTTI_PATH, 'sponsors.txt'), sep='|')
studies = pd.read_csv(os.path.join(CTTI_PATH, 'studies.txt'), sep='|')
stocks_df = pd.read_csv(SAVE_STOCKS_PATH)

stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], utc=True)

sponsors['name'] = sponsors['name'].str.lower()
stocks_df['name'] = stocks_df['name'].str.lower()

sponsors = sponsors[sponsors['name'].isin(set(stocks_df['name']))]
sponsors = pd.merge(sponsors, studies[['nct_id', 'completion_date']], on='nct_id', how='inner')
sponsors['completion_date'] = pd.to_datetime(sponsors['completion_date'], utc=True)

tqdm.pandas()

sponsor_stock_dict = dict(list(stocks_df.groupby('name')))
sponsors['Slope'] = sponsors.progress_apply(lambda row: calculate_sma_slope(sponsor_stock_dict[row['name']], row['name'], row['completion_date'], 5, 7), axis=1)
sponsors.dropna(subset=['Slope'], inplace=True)

/tmp/ipykernel_1304388/3471275543.py:9: DtypeWarning: Columns (46,47,48,53,65,68) have mixed types. Specify dtype option on import or set low_memory=False.
  studies = pd.read_csv(os.path.join(CTTI_PATH, 'studies.txt'), sep='|')


In [82]:
sponsors.shape
# sponsors.to_csv('slope_labels.csv')

(30873, 7)

In [84]:
# import matplotlib.pyplot as plt
# # Group slopes by their sign (positive or negative)
# slope_groups = sponsors.groupby(sponsors['Slope'] > 0)

# # Calculate statistics
# slope_stats = slope_groups['Slope'].agg(['count', 'mean', 'std', 'min', 'max'])

# # Plot distribution of positive and negative slopes
# plt.figure(figsize=(8, 6))
# slope_groups['Slope'].count().plot(kind='bar', color=['red' if group else 'blue' for group in slope_groups.groups.keys()])
# plt.xlabel('Slope Sign')
# plt.ylabel('Count')
# plt.title('Distribution of Positive and Negative Slopes')
# plt.xticks(ticks=[0, 1], labels=['Negative', 'Positive'], rotation=0)
# plt.show()

# # Print statistics
# print("Statistics of Positive and Negative Slopes:")
# print(slope_stats)